In [2]:
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi

# Authenticatation of Kaggle API
api = KaggleApi()
api.authenticate()

# dataset
dataset_path = "animals10"
if not os.path.exists(dataset_path):
    print("Downloading Animals-10 dataset...")
    api.dataset_download_files('alessiocorrado99/animals10', path=dataset_path, unzip=False)

# Extract the dataset
with zipfile.ZipFile(f"{dataset_path}/animals10.zip", 'r') as zip_ref:
    print("Extracting dataset...")
    zip_ref.extractall(dataset_path)

print("Dataset downloaded and extracted successfully.")


Dataset URL: https://www.kaggle.com/datasets/alessiocorrado99/animals10
Extracting dataset...
Dataset downloaded and extracted successfully.


In [3]:
import shutil
from sklearn.model_selection import train_test_split
import glob


original_data_dir = "animals10/raw-img/"
train_dir = "dataset/training_set/"
test_dir = "dataset/test_set/"

# Create directories
for category in os.listdir(original_data_dir):
    category_path = os.path.join(original_data_dir, category)
    if os.path.isdir(category_path):
        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        # Split into training and test sets
        images = glob.glob(f"{category_path}/*")
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Move files
        for image in train_images:
            shutil.move(image, os.path.join(train_dir, category))
        for image in test_images:
            shutil.move(image, os.path.join(test_dir, category))

print("Dataset organized into training_set and test_set.")


Dataset organized into training_set and test_set.


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Part 1 - Data Preprocessing

# Preprocess training set
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Scale images to into 0 to 1 range
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'  # Multi-class classification
)

# Preprocess test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Part 2 - Build CNN
cnn = tf.keras.models.Sequential()

# 1st Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# 2nd Convolution + Pooling
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

# Flatten
cnn.add(tf.keras.layers.Flatten())

# Fully Connected Layers
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))  # Hidden layer
cnn.add(tf.keras.layers.Dense(units=len(train_generator.class_indices), activation='softmax'))  # Output layer

# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN
cnn.fit(x=train_generator, validation_data=test_generator, epochs=30)

# Save the model after training
cnn.save('trained_model.h5')




Found 20938 images belonging to 10 classes.
Found 5241 images belonging to 10 classes.


C:\Users\Dell\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 214s 321ms/step - accuracy: 0.2959 - loss: 1.9973 - val_accuracy: 0.4272 - val_loss: 1.7584
Epoch 2/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 280s 428ms/step - accuracy: 0.5009 - loss: 1.4735 - val_accuracy: 0.4862 - val_loss: 1.5528
Epoch 3/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 118s 180ms/step - accuracy: 0.5667 - loss: 1.2865 - val_accuracy: 0.5415 - val_loss: 1.3144
Epoch 4/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 162s 248ms/step - accuracy: 0.6044 - loss: 1.1775 - val_accuracy: 0.5854 - val_loss: 1.2613
Epoch 5/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 172s 263ms/step - accuracy: 0.6293 - loss: 1.0936 - val_accuracy: 0.5753 - val_loss: 1.2953
Epoch 6/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 140s 168ms/step - accuracy: 0.6578 - loss: 1.0210 - val_accuracy: 0.5980 - val_loss: 1.2106
Epoch 7/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 109s 166ms/step - accuracy: 0.6749 - loss: 0.9704 - val_accuracy: 0.6369 - val_loss: 1.0882
Epoch 8/30
655/655 ━━━━━━━━━━━━━━━━━━━━ 109s 167ms/step - accuracy: 0.6875 -

In [8]:
import json

# Assuming train_generator has been defined in the code and includes class indices
class_indices = train_generator.class_indices

# Reverse the mapping to get class labels
class_labels = {v: k for k, v in class_indices.items()}

# Save the class labels to a JSON file
with open('dataset_structure.json', 'w') as f:
    json.dump(class_labels, f)

print("Dataset structure saved to 'dataset_structure.json'")


Dataset structure saved to 'dataset_structure.json'


In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import json

# Load the saved model
cnn = load_model('trained_model.h5')

# Load the class labels (class_names) from the saved JSON file
with open('dataset_structure.json', 'r') as f:
    class_names = json.load(f)

# Check the number of classes in class_names
'''print(f"Number of classes in class_names: {len(class_names)}")
print("Class names:", class_names)  # Print the class names for debugging'''

# Ensure the predicted class index is within bounds
if len(class_names) < 10:
    print("Warning: class_names list has fewer than 10 items. Check your dataset_structure.json.")

# Load and preprocess a single image for prediction
test_image = image.load_img(r'D:/animal classification/test1.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension

# Make prediction
result = cnn.predict(test_image)

# Check the shape of the result
#print("Prediction result:", result)

# Get the index of the predicted class
predicted_class = np.argmax(result)

# Verify predicted_class value
#print(f"Predicted class index: {predicted_class}")

# Ensure the predicted index is within the valid range
if str(predicted_class) in class_names:
    prediction = class_names[str(predicted_class)]  # Use the class index as a string key
    print(f"Predicted Class: {prediction}")
else:
    print("Error: Predicted class index is out of range!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Predicted Class: cavallo


In [11]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_test_data(test_set):
    X_test, y_test = [], []
    for images, labels in test_set:  # Assuming `test_set` is a generator
        X_test.extend(images)
        y_test.extend(labels)
    return np.array(X_test), np.array(y_test)

def evaluate_model(model, test_set):
    # Load test data
    print("Loading test data...")
    X_test, y_test = load_test_data(test_set)
    print(f"Loaded test data: {X_test.shape} images, {y_test.shape} labels")

    # Normalize if not already done by generator
    X_test = X_test / 255.0 if np.max(X_test) > 1 else X_test

    # Convert one-hot encoded labels to class indices
    y_test_classes = np.argmax(y_test, axis=1)

    # Get model predictions
    print("Running predictions...")
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Confusion Matrix
    cm = confusion_matrix(y_test_classes, y_pred_classes)
    print("Confusion Matrix:")
    print(cm)

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test_classes, y_pred_classes))

    # Accuracy Score
    accuracy = accuracy_score(y_test_classes, y_pred_classes)
    print(f"\nAccuracy Score: {accuracy * 100:.2f}%")


In [12]:
from tensorflow.keras.models import load_model
model = load_model(r'D:/animal classification/trained_model.h5')


In [13]:
test_datagen = ImageDataGenerator(rescale=1.0/255)
test_set = test_datagen.flow_from_directory(
    r'D:/animal classification/dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 5241 images belonging to 10 classes.


In [14]:
# Evaluate the model on the test set directly using batches
results = cnn.evaluate(test_set, verbose=1)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1] * 100:.2f}%")


164/164 ━━━━━━━━━━━━━━━━━━━━ 12s 68ms/step - accuracy: 0.6014 - loss: 1.4783
Test Loss: 1.4161070585250854
Test Accuracy: 63.56%
